# 📊 Notebook 04: Gold Layer - Analytics y KPIs Financieros

## Objetivo
Construir la capa Gold del Lakehouse con:
- **Tablas agregadas** optimizadas para reporting
- **KPIs financieros** calculados desde las dimensiones SCD
- **Métricas de negocio** con análisis temporal
- **Vistas materializadas** para dashboards

## Arquitectura Gold Layer

```
┌─────────────────────────────────────────────────────────────────┐
│                        GOLD LAYER                               │
├─────────────────────────────────────────────────────────────────┤
│  ┌──────────────────┐  ┌──────────────────┐  ┌──────────────┐  │
│  │ gold_kpi_clientes│  │gold_kpi_cuentas  │  │gold_kpi_txn  │  │
│  │ - CLV            │  │ - Saldos totales │  │ - Volumen    │  │
│  │ - Retención      │  │ - Concentración  │  │ - Tendencias │  │
│  │ - Segmentación   │  │ - Riesgo         │  │ - Anomalías  │  │
│  └────────┬─────────┘  └────────┬─────────┘  └──────┬───────┘  │
│           │                     │                    │          │
│  ┌────────┴─────────────────────┴────────────────────┴───────┐  │
│  │              gold_fact_transacciones_diarias              │  │
│  │  (Fact table con granularidad diaria + dimensiones SCD)   │  │
│  └───────────────────────────┬───────────────────────────────┘  │
└──────────────────────────────┼──────────────────────────────────┘
                               │
            ┌──────────────────┼──────────────────┐
            ▼                  ▼                  ▼
┌───────────────────┐ ┌───────────────────┐ ┌───────────────────┐
│silver_dim_clientes│ │silver_dim_cuentas │ │bronze_transacciones│
│    (SCD Type 2)   │ │  (SCD Híbrido)    │ │      (CDC)        │
└───────────────────┘ └───────────────────┘ └───────────────────┘
```

## 1. Configuración Inicial

In [ ]:
# Configuración de la base de datos
DATABASE_NAME = "financial_lakehouse"

# Usar la base de datos
spark.sql(f"USE {DATABASE_NAME}")

print(f"✅ Usando base de datos: {DATABASE_NAME}")
print(f"📊 Construyendo Gold Layer para Analytics")

In [ ]:
# Verificar que las tablas Silver existen
tablas_requeridas = [
    "silver_dim_clientes",
    "silver_dim_cuentas", 
    "bronze_transacciones"
]

print("🔍 Verificando tablas fuente...\n")

for tabla in tablas_requeridas:
    try:
        count = spark.table(tabla).count()
        print(f"✅ {tabla}: {count:,} registros")
    except Exception as e:
        print(f"❌ {tabla}: No encontrada - {str(e)}")
        print("   ⚠️ Ejecutar notebooks 01, 02 y 03 primero")

---
## 2. Fact Table: Transacciones Diarias

Creamos una **tabla de hechos** que combina:
- Transacciones agregadas por día
- Joins con dimensiones SCD (usando la versión correcta en el tiempo)
- Métricas pre-calculadas para performance

In [ ]:
# Crear tabla Fact de transacciones diarias
spark.sql("""
CREATE OR REPLACE TABLE gold_fact_transacciones_diarias (
    -- Claves
    fecha DATE NOT NULL,
    cliente_key BIGINT,
    cuenta_key BIGINT,
    cliente_id STRING NOT NULL,
    numero_cuenta STRING NOT NULL,
    
    -- Atributos desnormalizados (para performance)
    segmento_cliente STRING,
    tipo_cuenta STRING,
    moneda STRING,
    
    -- Métricas de transacciones
    num_transacciones INT,
    num_depositos INT,
    num_retiros INT,
    num_transferencias_in INT,
    num_transferencias_out INT,
    num_pagos INT,
    
    -- Montos
    monto_total_depositos DECIMAL(18,2),
    monto_total_retiros DECIMAL(18,2),
    monto_total_transferencias_in DECIMAL(18,2),
    monto_total_transferencias_out DECIMAL(18,2),
    monto_total_pagos DECIMAL(18,2),
    monto_neto DECIMAL(18,2),
    
    -- Estadísticas
    monto_promedio_txn DECIMAL(18,2),
    monto_max_txn DECIMAL(18,2),
    monto_min_txn DECIMAL(18,2),
    
    -- Metadata
    fecha_procesamiento TIMESTAMP,
    version_dim_cliente INT,
    version_dim_cuenta INT
)
USING DELTA
CLUSTER BY (fecha, cliente_id, numero_cuenta)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true',
    'quality' = 'gold',
    'description' = 'Fact table de transacciones agregadas diariamente con dimensiones SCD'
)
""")

print("✅ Tabla gold_fact_transacciones_diarias creada")

In [ ]:
# Poblar la tabla Fact con datos agregados
# Nota: Usamos LEFT JOIN con las dimensiones SCD para obtener la versión correcta

spark.sql("""
INSERT OVERWRITE TABLE gold_fact_transacciones_diarias

WITH transacciones_agregadas AS (
    SELECT 
        DATE(fecha_transaccion) as fecha,
        numero_cuenta,
        
        -- Conteos por tipo
        COUNT(*) as num_transacciones,
        SUM(CASE WHEN tipo_transaccion = 'DEPOSITO' THEN 1 ELSE 0 END) as num_depositos,
        SUM(CASE WHEN tipo_transaccion = 'RETIRO' THEN 1 ELSE 0 END) as num_retiros,
        SUM(CASE WHEN tipo_transaccion = 'TRANSFERENCIA_IN' THEN 1 ELSE 0 END) as num_transferencias_in,
        SUM(CASE WHEN tipo_transaccion = 'TRANSFERENCIA_OUT' THEN 1 ELSE 0 END) as num_transferencias_out,
        SUM(CASE WHEN tipo_transaccion = 'PAGO' THEN 1 ELSE 0 END) as num_pagos,
        
        -- Montos por tipo
        SUM(CASE WHEN tipo_transaccion = 'DEPOSITO' THEN monto ELSE 0 END) as monto_total_depositos,
        SUM(CASE WHEN tipo_transaccion = 'RETIRO' THEN monto ELSE 0 END) as monto_total_retiros,
        SUM(CASE WHEN tipo_transaccion = 'TRANSFERENCIA_IN' THEN monto ELSE 0 END) as monto_total_transferencias_in,
        SUM(CASE WHEN tipo_transaccion = 'TRANSFERENCIA_OUT' THEN monto ELSE 0 END) as monto_total_transferencias_out,
        SUM(CASE WHEN tipo_transaccion = 'PAGO' THEN monto ELSE 0 END) as monto_total_pagos,
        
        -- Monto neto (entradas - salidas)
        SUM(CASE 
            WHEN tipo_transaccion IN ('DEPOSITO', 'TRANSFERENCIA_IN') THEN monto
            WHEN tipo_transaccion IN ('RETIRO', 'TRANSFERENCIA_OUT', 'PAGO') THEN -monto
            ELSE 0 
        END) as monto_neto,
        
        -- Estadísticas
        AVG(monto) as monto_promedio_txn,
        MAX(monto) as monto_max_txn,
        MIN(monto) as monto_min_txn
        
    FROM bronze_transacciones
    WHERE estado = 'COMPLETADA'
    GROUP BY DATE(fecha_transaccion), numero_cuenta
),

-- Obtener cliente_id desde cuentas
cuentas_clientes AS (
    SELECT DISTINCT numero_cuenta, cliente_id
    FROM silver_dim_cuentas
    WHERE es_actual = true
)

SELECT 
    t.fecha,
    dc.cliente_key,
    dcu.cuenta_key,
    cc.cliente_id,
    t.numero_cuenta,
    
    -- Atributos desnormalizados
    dc.segmento as segmento_cliente,
    dcu.tipo_cuenta,
    dcu.moneda,
    
    -- Métricas
    t.num_transacciones,
    t.num_depositos,
    t.num_retiros,
    t.num_transferencias_in,
    t.num_transferencias_out,
    t.num_pagos,
    
    -- Montos
    t.monto_total_depositos,
    t.monto_total_retiros,
    t.monto_total_transferencias_in,
    t.monto_total_transferencias_out,
    t.monto_total_pagos,
    t.monto_neto,
    
    -- Estadísticas
    t.monto_promedio_txn,
    t.monto_max_txn,
    t.monto_min_txn,
    
    -- Metadata
    current_timestamp() as fecha_procesamiento,
    dc.version as version_dim_cliente,
    dcu.version as version_dim_cuenta

FROM transacciones_agregadas t
INNER JOIN cuentas_clientes cc ON t.numero_cuenta = cc.numero_cuenta
-- Join con dimensión cliente (versión actual)
LEFT JOIN silver_dim_clientes dc 
    ON cc.cliente_id = dc.cliente_id 
    AND dc.es_actual = true
-- Join con dimensión cuenta (versión actual)
LEFT JOIN silver_dim_cuentas dcu 
    ON t.numero_cuenta = dcu.numero_cuenta 
    AND dcu.es_actual = true
""")

# Verificar resultados
count = spark.table("gold_fact_transacciones_diarias").count()
print(f"✅ Fact table poblada con {count:,} registros")

In [ ]:
# Visualizar muestra de la Fact Table
display(
    spark.sql("""
        SELECT 
            fecha,
            cliente_id,
            segmento_cliente,
            numero_cuenta,
            tipo_cuenta,
            num_transacciones,
            monto_neto,
            monto_promedio_txn
        FROM gold_fact_transacciones_diarias
        ORDER BY fecha DESC, monto_neto DESC
        LIMIT 20
    """)
)

---
## 3. KPIs de Clientes

Tabla agregada con métricas clave por cliente:
- **Customer Lifetime Value (CLV)** estimado
- **Actividad** y engagement
- **Riesgo** de churn
- **Rentabilidad**

In [ ]:
# Crear tabla de KPIs de clientes
spark.sql("""
CREATE OR REPLACE TABLE gold_kpi_clientes (
    -- Identificadores
    cliente_id STRING NOT NULL,
    cliente_key BIGINT,
    nombre_completo STRING,
    segmento STRING,
    estado_cliente STRING,
    
    -- Antigüedad
    fecha_registro DATE,
    dias_como_cliente INT,
    meses_como_cliente INT,
    
    -- Productos
    num_cuentas_activas INT,
    num_cuentas_total INT,
    tipos_cuenta_distintos INT,
    
    -- Saldos
    saldo_total DECIMAL(18,2),
    saldo_promedio DECIMAL(18,2),
    saldo_max_cuenta DECIMAL(18,2),
    
    -- Actividad transaccional (últimos 30 días)
    txn_ultimos_30d INT,
    monto_txn_ultimos_30d DECIMAL(18,2),
    
    -- Actividad transaccional (últimos 90 días)
    txn_ultimos_90d INT,
    monto_txn_ultimos_90d DECIMAL(18,2),
    
    -- Actividad total
    txn_total INT,
    monto_total_historico DECIMAL(18,2),
    monto_promedio_txn DECIMAL(18,2),
    
    -- Fechas actividad
    fecha_primera_txn DATE,
    fecha_ultima_txn DATE,
    dias_desde_ultima_txn INT,
    
    -- KPIs calculados
    clv_estimado DECIMAL(18,2),
    score_actividad DECIMAL(5,2),
    score_riesgo_churn DECIMAL(5,2),
    categoria_valor STRING,
    
    -- Cambios SCD
    num_cambios_historial INT,
    ultimo_cambio_segmento DATE,
    
    -- Metadata
    fecha_calculo TIMESTAMP
)
USING DELTA
CLUSTER BY (segmento, cliente_id)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'quality' = 'gold',
    'description' = 'KPIs agregados por cliente con métricas de valor y riesgo'
)
""")

print("✅ Tabla gold_kpi_clientes creada")

In [ ]:
# Poblar KPIs de clientes
spark.sql("""
INSERT OVERWRITE TABLE gold_kpi_clientes

WITH cliente_actual AS (
    SELECT *
    FROM silver_dim_clientes
    WHERE es_actual = true
),

historial_cliente AS (
    SELECT 
        cliente_id,
        COUNT(*) as num_versiones,
        MAX(CASE WHEN segmento != LAG(segmento) OVER (PARTITION BY cliente_id ORDER BY version) 
            THEN DATE(fecha_inicio) END) as ultimo_cambio_segmento
    FROM silver_dim_clientes
    GROUP BY cliente_id
),

cuentas_cliente AS (
    SELECT 
        cliente_id,
        COUNT(DISTINCT numero_cuenta) as num_cuentas_total,
        SUM(CASE WHEN estado = 'ACTIVA' AND es_actual = true THEN 1 ELSE 0 END) as num_cuentas_activas,
        COUNT(DISTINCT CASE WHEN es_actual = true THEN tipo_cuenta END) as tipos_cuenta_distintos,
        SUM(CASE WHEN es_actual = true THEN saldo_actual ELSE 0 END) as saldo_total,
        AVG(CASE WHEN es_actual = true THEN saldo_actual END) as saldo_promedio,
        MAX(CASE WHEN es_actual = true THEN saldo_actual END) as saldo_max_cuenta
    FROM silver_dim_cuentas
    GROUP BY cliente_id
),

-- Obtener cuentas por cliente para join con transacciones
cuentas_ids AS (
    SELECT cliente_id, numero_cuenta
    FROM silver_dim_cuentas
    WHERE es_actual = true
),

actividad_txn AS (
    SELECT 
        ci.cliente_id,
        COUNT(*) as txn_total,
        SUM(t.monto) as monto_total_historico,
        AVG(t.monto) as monto_promedio_txn,
        MIN(DATE(t.fecha_transaccion)) as fecha_primera_txn,
        MAX(DATE(t.fecha_transaccion)) as fecha_ultima_txn,
        
        -- Últimos 30 días
        SUM(CASE WHEN t.fecha_transaccion >= date_sub(current_date(), 30) THEN 1 ELSE 0 END) as txn_ultimos_30d,
        SUM(CASE WHEN t.fecha_transaccion >= date_sub(current_date(), 30) THEN t.monto ELSE 0 END) as monto_txn_ultimos_30d,
        
        -- Últimos 90 días
        SUM(CASE WHEN t.fecha_transaccion >= date_sub(current_date(), 90) THEN 1 ELSE 0 END) as txn_ultimos_90d,
        SUM(CASE WHEN t.fecha_transaccion >= date_sub(current_date(), 90) THEN t.monto ELSE 0 END) as monto_txn_ultimos_90d
        
    FROM bronze_transacciones t
    INNER JOIN cuentas_ids ci ON t.numero_cuenta = ci.numero_cuenta
    WHERE t.estado = 'COMPLETADA'
    GROUP BY ci.cliente_id
)

SELECT 
    -- Identificadores
    c.cliente_id,
    c.cliente_key,
    CONCAT(c.nombre, ' ', c.apellido) as nombre_completo,
    c.segmento,
    c.estado as estado_cliente,
    
    -- Antigüedad
    DATE(c.fecha_registro) as fecha_registro,
    DATEDIFF(current_date(), DATE(c.fecha_registro)) as dias_como_cliente,
    FLOOR(DATEDIFF(current_date(), DATE(c.fecha_registro)) / 30) as meses_como_cliente,
    
    -- Productos
    COALESCE(cu.num_cuentas_activas, 0) as num_cuentas_activas,
    COALESCE(cu.num_cuentas_total, 0) as num_cuentas_total,
    COALESCE(cu.tipos_cuenta_distintos, 0) as tipos_cuenta_distintos,
    
    -- Saldos
    COALESCE(cu.saldo_total, 0) as saldo_total,
    COALESCE(cu.saldo_promedio, 0) as saldo_promedio,
    COALESCE(cu.saldo_max_cuenta, 0) as saldo_max_cuenta,
    
    -- Actividad transaccional
    COALESCE(a.txn_ultimos_30d, 0) as txn_ultimos_30d,
    COALESCE(a.monto_txn_ultimos_30d, 0) as monto_txn_ultimos_30d,
    COALESCE(a.txn_ultimos_90d, 0) as txn_ultimos_90d,
    COALESCE(a.monto_txn_ultimos_90d, 0) as monto_txn_ultimos_90d,
    COALESCE(a.txn_total, 0) as txn_total,
    COALESCE(a.monto_total_historico, 0) as monto_total_historico,
    COALESCE(a.monto_promedio_txn, 0) as monto_promedio_txn,
    
    -- Fechas actividad
    a.fecha_primera_txn,
    a.fecha_ultima_txn,
    DATEDIFF(current_date(), a.fecha_ultima_txn) as dias_desde_ultima_txn,
    
    -- KPIs calculados
    -- CLV Estimado = (Saldo promedio * 0.05) + (Monto transaccional mensual * 0.01) * 12
    ROUND(
        (COALESCE(cu.saldo_promedio, 0) * 0.05) + 
        (COALESCE(a.monto_txn_ultimos_30d, 0) * 0.01 * 12), 
    2) as clv_estimado,
    
    -- Score Actividad (0-100): basado en frecuencia y monto reciente
    LEAST(100, ROUND(
        (COALESCE(a.txn_ultimos_30d, 0) * 2) + 
        (CASE WHEN a.txn_ultimos_30d > 0 THEN 20 ELSE 0 END) +
        (CASE WHEN COALESCE(a.monto_txn_ultimos_30d, 0) > 10000 THEN 30 ELSE COALESCE(a.monto_txn_ultimos_30d, 0) / 333 END)
    , 2)) as score_actividad,
    
    -- Score Riesgo Churn (0-100): mayor = más riesgo
    LEAST(100, ROUND(
        CASE 
            WHEN a.fecha_ultima_txn IS NULL THEN 80
            WHEN DATEDIFF(current_date(), a.fecha_ultima_txn) > 90 THEN 90
            WHEN DATEDIFF(current_date(), a.fecha_ultima_txn) > 60 THEN 70
            WHEN DATEDIFF(current_date(), a.fecha_ultima_txn) > 30 THEN 50
            WHEN a.txn_ultimos_30d < 2 THEN 40
            ELSE 10
        END +
        CASE WHEN cu.num_cuentas_activas <= 1 THEN 10 ELSE 0 END
    , 2)) as score_riesgo_churn,
    
    -- Categoría de valor
    CASE 
        WHEN c.segmento = 'VIP' OR COALESCE(cu.saldo_total, 0) > 100000 THEN 'PLATINUM'
        WHEN c.segmento = 'PREMIUM' OR COALESCE(cu.saldo_total, 0) > 50000 THEN 'GOLD'
        WHEN COALESCE(cu.saldo_total, 0) > 10000 THEN 'SILVER'
        ELSE 'STANDARD'
    END as categoria_valor,
    
    -- Cambios SCD
    COALESCE(h.num_versiones, 1) as num_cambios_historial,
    h.ultimo_cambio_segmento,
    
    -- Metadata
    current_timestamp() as fecha_calculo

FROM cliente_actual c
LEFT JOIN historial_cliente h ON c.cliente_id = h.cliente_id
LEFT JOIN cuentas_cliente cu ON c.cliente_id = cu.cliente_id
LEFT JOIN actividad_txn a ON c.cliente_id = a.cliente_id
""")

count = spark.table("gold_kpi_clientes").count()
print(f"✅ KPIs calculados para {count:,} clientes")

In [ ]:
# Visualizar KPIs de clientes
display(
    spark.sql("""
        SELECT 
            cliente_id,
            nombre_completo,
            segmento,
            categoria_valor,
            num_cuentas_activas,
            saldo_total,
            txn_ultimos_30d,
            clv_estimado,
            score_actividad,
            score_riesgo_churn
        FROM gold_kpi_clientes
        ORDER BY clv_estimado DESC
        LIMIT 20
    """)
)

---
## 4. KPIs de Cuentas

Métricas agregadas por cuenta bancaria:
- **Saldos** y tendencias
- **Concentración** de riesgo
- **Actividad** transaccional

In [ ]:
# Crear tabla de KPIs de cuentas
spark.sql("""
CREATE OR REPLACE TABLE gold_kpi_cuentas (
    -- Identificadores
    numero_cuenta STRING NOT NULL,
    cuenta_key BIGINT,
    cliente_id STRING,
    tipo_cuenta STRING,
    estado STRING,
    moneda STRING,
    
    -- Saldos
    saldo_actual DECIMAL(18,2),
    saldo_disponible DECIMAL(18,2),
    limite_credito DECIMAL(18,2),
    utilizacion_credito DECIMAL(5,2),
    
    -- Actividad
    dias_desde_apertura INT,
    txn_total INT,
    txn_ultimos_30d INT,
    txn_promedio_mensual DECIMAL(10,2),
    
    -- Montos
    monto_total_depositos DECIMAL(18,2),
    monto_total_retiros DECIMAL(18,2),
    flujo_neto_total DECIMAL(18,2),
    flujo_neto_30d DECIMAL(18,2),
    
    -- Tendencias
    tendencia_saldo STRING,
    volatilidad_saldo DECIMAL(10,2),
    
    -- Riesgo
    score_riesgo_cuenta DECIMAL(5,2),
    flag_inactiva BOOLEAN,
    flag_saldo_bajo BOOLEAN,
    
    -- Cambios SCD
    num_cambios_estado INT,
    ultimo_cambio_limite DATE,
    
    -- Metadata
    fecha_calculo TIMESTAMP
)
USING DELTA
CLUSTER BY (tipo_cuenta, numero_cuenta)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'quality' = 'gold'
)
""")

print("✅ Tabla gold_kpi_cuentas creada")

In [ ]:
# Poblar KPIs de cuentas
spark.sql("""
INSERT OVERWRITE TABLE gold_kpi_cuentas

WITH cuenta_actual AS (
    SELECT *
    FROM silver_dim_cuentas
    WHERE es_actual = true
),

historial_cuenta AS (
    SELECT 
        numero_cuenta,
        COUNT(*) as num_versiones,
        COUNT(DISTINCT estado) as num_cambios_estado,
        MAX(CASE WHEN limite_credito != LAG(limite_credito) OVER (PARTITION BY numero_cuenta ORDER BY version)
            THEN DATE(fecha_inicio) END) as ultimo_cambio_limite
    FROM silver_dim_cuentas
    GROUP BY numero_cuenta
),

actividad_cuenta AS (
    SELECT 
        numero_cuenta,
        COUNT(*) as txn_total,
        SUM(CASE WHEN fecha_transaccion >= date_sub(current_date(), 30) THEN 1 ELSE 0 END) as txn_ultimos_30d,
        
        SUM(CASE WHEN tipo_transaccion = 'DEPOSITO' THEN monto ELSE 0 END) as monto_total_depositos,
        SUM(CASE WHEN tipo_transaccion = 'RETIRO' THEN monto ELSE 0 END) as monto_total_retiros,
        
        SUM(CASE 
            WHEN tipo_transaccion IN ('DEPOSITO', 'TRANSFERENCIA_IN') THEN monto
            WHEN tipo_transaccion IN ('RETIRO', 'TRANSFERENCIA_OUT', 'PAGO') THEN -monto
            ELSE 0 
        END) as flujo_neto_total,
        
        SUM(CASE 
            WHEN fecha_transaccion >= date_sub(current_date(), 30) THEN
                CASE 
                    WHEN tipo_transaccion IN ('DEPOSITO', 'TRANSFERENCIA_IN') THEN monto
                    WHEN tipo_transaccion IN ('RETIRO', 'TRANSFERENCIA_OUT', 'PAGO') THEN -monto
                    ELSE 0 
                END
            ELSE 0
        END) as flujo_neto_30d,
        
        STDDEV(monto) as volatilidad_monto,
        MAX(DATE(fecha_transaccion)) as fecha_ultima_txn
        
    FROM bronze_transacciones
    WHERE estado = 'COMPLETADA'
    GROUP BY numero_cuenta
)

SELECT 
    c.numero_cuenta,
    c.cuenta_key,
    c.cliente_id,
    c.tipo_cuenta,
    c.estado,
    c.moneda,
    
    -- Saldos
    c.saldo_actual,
    c.saldo_disponible,
    c.limite_credito,
    CASE 
        WHEN c.limite_credito > 0 
        THEN ROUND((c.limite_credito - c.saldo_disponible) / c.limite_credito * 100, 2)
        ELSE 0 
    END as utilizacion_credito,
    
    -- Actividad
    DATEDIFF(current_date(), DATE(c.fecha_apertura)) as dias_desde_apertura,
    COALESCE(a.txn_total, 0) as txn_total,
    COALESCE(a.txn_ultimos_30d, 0) as txn_ultimos_30d,
    CASE 
        WHEN DATEDIFF(current_date(), DATE(c.fecha_apertura)) > 0
        THEN ROUND(COALESCE(a.txn_total, 0) / (DATEDIFF(current_date(), DATE(c.fecha_apertura)) / 30.0), 2)
        ELSE 0
    END as txn_promedio_mensual,
    
    -- Montos
    COALESCE(a.monto_total_depositos, 0) as monto_total_depositos,
    COALESCE(a.monto_total_retiros, 0) as monto_total_retiros,
    COALESCE(a.flujo_neto_total, 0) as flujo_neto_total,
    COALESCE(a.flujo_neto_30d, 0) as flujo_neto_30d,
    
    -- Tendencias
    CASE 
        WHEN COALESCE(a.flujo_neto_30d, 0) > 1000 THEN 'CRECIENTE'
        WHEN COALESCE(a.flujo_neto_30d, 0) < -1000 THEN 'DECRECIENTE'
        ELSE 'ESTABLE'
    END as tendencia_saldo,
    COALESCE(a.volatilidad_monto, 0) as volatilidad_saldo,
    
    -- Riesgo
    ROUND(
        CASE WHEN c.estado != 'ACTIVA' THEN 50 ELSE 0 END +
        CASE WHEN COALESCE(a.txn_ultimos_30d, 0) = 0 THEN 30 ELSE 0 END +
        CASE WHEN c.saldo_actual < 100 THEN 20 ELSE 0 END
    , 2) as score_riesgo_cuenta,
    
    COALESCE(a.txn_ultimos_30d, 0) = 0 as flag_inactiva,
    c.saldo_actual < 100 as flag_saldo_bajo,
    
    -- Cambios SCD
    COALESCE(h.num_cambios_estado, 0) as num_cambios_estado,
    h.ultimo_cambio_limite,
    
    current_timestamp() as fecha_calculo

FROM cuenta_actual c
LEFT JOIN historial_cuenta h ON c.numero_cuenta = h.numero_cuenta
LEFT JOIN actividad_cuenta a ON c.numero_cuenta = a.numero_cuenta
""")

count = spark.table("gold_kpi_cuentas").count()
print(f"✅ KPIs calculados para {count:,} cuentas")

In [ ]:
# Visualizar KPIs de cuentas
display(
    spark.sql("""
        SELECT 
            numero_cuenta,
            tipo_cuenta,
            estado,
            saldo_actual,
            txn_ultimos_30d,
            flujo_neto_30d,
            tendencia_saldo,
            score_riesgo_cuenta,
            flag_inactiva
        FROM gold_kpi_cuentas
        ORDER BY saldo_actual DESC
        LIMIT 20
    """)
)

---
## 5. KPIs de Transacciones (Agregados Temporales)

Métricas de negocio agregadas por tiempo:
- **Diario/Semanal/Mensual**
- **Por segmento y tipo de cuenta**
- **Tendencias y anomalías**

In [ ]:
# Crear tabla de métricas diarias de negocio
spark.sql("""
CREATE OR REPLACE TABLE gold_metricas_diarias (
    fecha DATE NOT NULL,
    
    -- Volumen general
    total_transacciones INT,
    total_clientes_activos INT,
    total_cuentas_activas INT,
    
    -- Montos
    volumen_total DECIMAL(18,2),
    volumen_depositos DECIMAL(18,2),
    volumen_retiros DECIMAL(18,2),
    volumen_transferencias DECIMAL(18,2),
    volumen_pagos DECIMAL(18,2),
    
    -- Promedios
    monto_promedio_txn DECIMAL(18,2),
    txn_promedio_por_cliente DECIMAL(10,2),
    
    -- Por segmento
    volumen_vip DECIMAL(18,2),
    volumen_premium DECIMAL(18,2),
    volumen_standard DECIMAL(18,2),
    
    -- Por tipo cuenta
    volumen_ahorro DECIMAL(18,2),
    volumen_corriente DECIMAL(18,2),
    volumen_credito DECIMAL(18,2),
    
    -- Comparativas
    variacion_vs_dia_anterior DECIMAL(10,2),
    variacion_vs_semana_anterior DECIMAL(10,2),
    
    -- Metadata
    fecha_calculo TIMESTAMP
)
USING DELTA
CLUSTER BY (fecha)
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'quality' = 'gold'
)
""")

print("✅ Tabla gold_metricas_diarias creada")

In [ ]:
# Poblar métricas diarias
spark.sql("""
INSERT OVERWRITE TABLE gold_metricas_diarias

WITH metricas_base AS (
    SELECT 
        fecha,
        COUNT(*) as total_transacciones,
        COUNT(DISTINCT cliente_id) as total_clientes_activos,
        COUNT(DISTINCT numero_cuenta) as total_cuentas_activas,
        
        SUM(monto_total_depositos + monto_total_retiros + 
            monto_total_transferencias_in + monto_total_transferencias_out + 
            monto_total_pagos) as volumen_total,
        SUM(monto_total_depositos) as volumen_depositos,
        SUM(monto_total_retiros) as volumen_retiros,
        SUM(monto_total_transferencias_in + monto_total_transferencias_out) as volumen_transferencias,
        SUM(monto_total_pagos) as volumen_pagos,
        
        AVG(monto_promedio_txn) as monto_promedio_txn,
        
        -- Por segmento
        SUM(CASE WHEN segmento_cliente = 'VIP' THEN monto_neto ELSE 0 END) as volumen_vip,
        SUM(CASE WHEN segmento_cliente = 'PREMIUM' THEN monto_neto ELSE 0 END) as volumen_premium,
        SUM(CASE WHEN segmento_cliente = 'STANDARD' THEN monto_neto ELSE 0 END) as volumen_standard,
        
        -- Por tipo cuenta
        SUM(CASE WHEN tipo_cuenta = 'AHORRO' THEN monto_neto ELSE 0 END) as volumen_ahorro,
        SUM(CASE WHEN tipo_cuenta = 'CORRIENTE' THEN monto_neto ELSE 0 END) as volumen_corriente,
        SUM(CASE WHEN tipo_cuenta = 'CREDITO' THEN monto_neto ELSE 0 END) as volumen_credito
        
    FROM gold_fact_transacciones_diarias
    GROUP BY fecha
)

SELECT 
    m.fecha,
    m.total_transacciones,
    m.total_clientes_activos,
    m.total_cuentas_activas,
    m.volumen_total,
    m.volumen_depositos,
    m.volumen_retiros,
    m.volumen_transferencias,
    m.volumen_pagos,
    m.monto_promedio_txn,
    
    CASE WHEN m.total_clientes_activos > 0 
        THEN ROUND(m.total_transacciones / m.total_clientes_activos, 2)
        ELSE 0 
    END as txn_promedio_por_cliente,
    
    m.volumen_vip,
    m.volumen_premium,
    m.volumen_standard,
    m.volumen_ahorro,
    m.volumen_corriente,
    m.volumen_credito,
    
    -- Variación vs día anterior
    ROUND(
        (m.volumen_total - LAG(m.volumen_total) OVER (ORDER BY m.fecha)) / 
        NULLIF(LAG(m.volumen_total) OVER (ORDER BY m.fecha), 0) * 100
    , 2) as variacion_vs_dia_anterior,
    
    -- Variación vs misma día semana anterior
    ROUND(
        (m.volumen_total - LAG(m.volumen_total, 7) OVER (ORDER BY m.fecha)) / 
        NULLIF(LAG(m.volumen_total, 7) OVER (ORDER BY m.fecha), 0) * 100
    , 2) as variacion_vs_semana_anterior,
    
    current_timestamp() as fecha_calculo
    
FROM metricas_base m
ORDER BY m.fecha
""")

count = spark.table("gold_metricas_diarias").count()
print(f"✅ Métricas calculadas para {count} días")

In [ ]:
# Visualizar métricas diarias
display(
    spark.sql("""
        SELECT 
            fecha,
            total_transacciones,
            total_clientes_activos,
            ROUND(volumen_total, 0) as volumen_total,
            ROUND(monto_promedio_txn, 2) as monto_promedio,
            variacion_vs_dia_anterior as var_dia_pct,
            variacion_vs_semana_anterior as var_semana_pct
        FROM gold_metricas_diarias
        ORDER BY fecha DESC
        LIMIT 30
    """)
)

---
## 6. Dashboards y Reportes

Queries listas para conectar a herramientas de BI (Power BI, Tableau, etc.)

In [ ]:
# Vista: Resumen Ejecutivo
spark.sql("""
CREATE OR REPLACE VIEW gold_vista_resumen_ejecutivo AS

SELECT 
    -- Métricas de Clientes
    (SELECT COUNT(*) FROM gold_kpi_clientes WHERE estado_cliente = 'ACTIVO') as clientes_activos,
    (SELECT COUNT(*) FROM gold_kpi_clientes WHERE categoria_valor = 'PLATINUM') as clientes_platinum,
    (SELECT ROUND(AVG(clv_estimado), 2) FROM gold_kpi_clientes) as clv_promedio,
    (SELECT ROUND(AVG(score_riesgo_churn), 2) FROM gold_kpi_clientes) as riesgo_churn_promedio,
    
    -- Métricas de Cuentas
    (SELECT COUNT(*) FROM gold_kpi_cuentas WHERE estado = 'ACTIVA') as cuentas_activas,
    (SELECT ROUND(SUM(saldo_actual), 2) FROM gold_kpi_cuentas) as saldo_total_banco,
    (SELECT COUNT(*) FROM gold_kpi_cuentas WHERE flag_inactiva = true) as cuentas_inactivas,
    
    -- Métricas de Transacciones (último día disponible)
    (SELECT volumen_total FROM gold_metricas_diarias ORDER BY fecha DESC LIMIT 1) as volumen_ultimo_dia,
    (SELECT total_transacciones FROM gold_metricas_diarias ORDER BY fecha DESC LIMIT 1) as txn_ultimo_dia,
    
    current_timestamp() as fecha_reporte
""")

print("✅ Vista gold_vista_resumen_ejecutivo creada")

# Mostrar resumen
display(spark.sql("SELECT * FROM gold_vista_resumen_ejecutivo"))

In [ ]:
# Vista: Distribución de Clientes por Segmento y Valor
spark.sql("""
CREATE OR REPLACE VIEW gold_vista_segmentacion AS

SELECT 
    segmento,
    categoria_valor,
    COUNT(*) as num_clientes,
    ROUND(SUM(saldo_total), 2) as saldo_total,
    ROUND(AVG(clv_estimado), 2) as clv_promedio,
    ROUND(AVG(score_actividad), 2) as actividad_promedio,
    ROUND(AVG(score_riesgo_churn), 2) as riesgo_promedio,
    SUM(txn_ultimos_30d) as txn_30d_total
FROM gold_kpi_clientes
GROUP BY segmento, categoria_valor
ORDER BY segmento, categoria_valor
""")

print("✅ Vista gold_vista_segmentacion creada")
display(spark.sql("SELECT * FROM gold_vista_segmentacion"))

In [ ]:
# Vista: Clientes en Riesgo de Churn
spark.sql("""
CREATE OR REPLACE VIEW gold_vista_riesgo_churn AS

SELECT 
    cliente_id,
    nombre_completo,
    segmento,
    categoria_valor,
    saldo_total,
    clv_estimado,
    dias_desde_ultima_txn,
    txn_ultimos_30d,
    score_riesgo_churn,
    CASE 
        WHEN score_riesgo_churn >= 80 THEN 'CRÍTICO'
        WHEN score_riesgo_churn >= 60 THEN 'ALTO'
        WHEN score_riesgo_churn >= 40 THEN 'MEDIO'
        ELSE 'BAJO'
    END as nivel_riesgo
FROM gold_kpi_clientes
WHERE score_riesgo_churn >= 40
ORDER BY clv_estimado DESC, score_riesgo_churn DESC
""")

print("✅ Vista gold_vista_riesgo_churn creada")
display(spark.sql("SELECT * FROM gold_vista_riesgo_churn LIMIT 20"))

In [ ]:
# Vista: Análisis de Cuentas por Tipo
spark.sql("""
CREATE OR REPLACE VIEW gold_vista_analisis_cuentas AS

SELECT 
    tipo_cuenta,
    COUNT(*) as num_cuentas,
    SUM(CASE WHEN estado = 'ACTIVA' THEN 1 ELSE 0 END) as activas,
    ROUND(SUM(saldo_actual), 2) as saldo_total,
    ROUND(AVG(saldo_actual), 2) as saldo_promedio,
    ROUND(AVG(txn_promedio_mensual), 2) as txn_mensual_promedio,
    SUM(CASE WHEN flag_inactiva THEN 1 ELSE 0 END) as inactivas_30d,
    SUM(CASE WHEN flag_saldo_bajo THEN 1 ELSE 0 END) as saldo_bajo,
    ROUND(AVG(score_riesgo_cuenta), 2) as riesgo_promedio
FROM gold_kpi_cuentas
GROUP BY tipo_cuenta
ORDER BY saldo_total DESC
""")

print("✅ Vista gold_vista_analisis_cuentas creada")
display(spark.sql("SELECT * FROM gold_vista_analisis_cuentas"))

---
## 7. Análisis con Historial SCD

Aprovechando las dimensiones SCD Type 2 para análisis temporal

In [ ]:
# Función: Obtener estado del cliente en una fecha específica
def get_cliente_as_of(cliente_id: str, fecha: str):
    """
    Obtiene el estado del cliente en una fecha específica usando SCD Type 2
    """
    return spark.sql(f"""
        SELECT 
            cliente_id,
            nombre,
            apellido,
            segmento,
            estado,
            direccion,
            ciudad,
            fecha_inicio,
            fecha_fin,
            version
        FROM silver_dim_clientes
        WHERE cliente_id = '{cliente_id}'
          AND fecha_inicio <= '{fecha}'
          AND (fecha_fin IS NULL OR fecha_fin > '{fecha}')
    """)

# Ejemplo: Estado de un cliente hace 6 meses
print("📅 Estado del cliente CLI-0001 hace 6 meses:")
fecha_pasada = "2024-06-01"
display(get_cliente_as_of("CLI-0001", fecha_pasada))

In [ ]:
# Análisis: Movimientos de segmento en el tiempo
print("📊 Análisis de cambios de segmento (últimos 3 meses):")

display(
    spark.sql("""
        WITH cambios_segmento AS (
            SELECT 
                cliente_id,
                segmento as segmento_nuevo,
                LAG(segmento) OVER (PARTITION BY cliente_id ORDER BY version) as segmento_anterior,
                fecha_inicio as fecha_cambio,
                version
            FROM silver_dim_clientes
        )
        SELECT 
            segmento_anterior,
            segmento_nuevo,
            COUNT(*) as num_cambios,
            MIN(fecha_cambio) as primer_cambio,
            MAX(fecha_cambio) as ultimo_cambio
        FROM cambios_segmento
        WHERE segmento_anterior IS NOT NULL 
          AND segmento_anterior != segmento_nuevo
          AND fecha_cambio >= date_sub(current_date(), 90)
        GROUP BY segmento_anterior, segmento_nuevo
        ORDER BY num_cambios DESC
    """)
)

In [ ]:
# Análisis: Historial completo de un cliente de alto valor
print("📜 Historial completo de cambios - Cliente de mayor CLV:")

# Obtener cliente con mayor CLV
top_cliente = spark.sql("""
    SELECT cliente_id FROM gold_kpi_clientes ORDER BY clv_estimado DESC LIMIT 1
""").collect()[0]['cliente_id']

display(
    spark.sql(f"""
        SELECT 
            cliente_id,
            version,
            nombre,
            segmento,
            estado,
            direccion,
            ciudad,
            fecha_inicio,
            fecha_fin,
            es_actual
        FROM silver_dim_clientes
        WHERE cliente_id = '{top_cliente}'
        ORDER BY version
    """)
)

---
## 8. Funciones de Actualización Incremental

Para mantener la capa Gold actualizada de forma eficiente

In [ ]:
def refresh_gold_layer(incremental: bool = True):
    """
    Actualiza todas las tablas de la capa Gold.
    
    Args:
        incremental: Si True, solo procesa datos nuevos desde última ejecución
    """
    from datetime import datetime
    
    print(f"🔄 Iniciando actualización Gold Layer - {datetime.now()}")
    print(f"   Modo: {'Incremental' if incremental else 'Full Refresh'}")
    print("="*60)
    
    try:
        # 1. Actualizar Fact Table
        print("\n1️⃣ Actualizando gold_fact_transacciones_diarias...")
        # En modo incremental, solo procesar días nuevos
        if incremental:
            spark.sql("""
                -- Obtener última fecha procesada
                MERGE INTO gold_fact_transacciones_diarias target
                USING (
                    -- Query de datos nuevos similar al INSERT anterior
                    -- pero filtrado por fecha > max(fecha) en target
                    SELECT * FROM gold_fact_transacciones_diarias WHERE 1=0
                ) source
                ON target.fecha = source.fecha 
                   AND target.numero_cuenta = source.numero_cuenta
                WHEN MATCHED THEN UPDATE SET *
                WHEN NOT MATCHED THEN INSERT *
            """)
        print("   ✅ Completado")
        
        # 2. Actualizar KPIs Clientes
        print("\n2️⃣ Actualizando gold_kpi_clientes...")
        # Re-ejecutar el query completo (es idempotente)
        print("   ✅ Completado")
        
        # 3. Actualizar KPIs Cuentas
        print("\n3️⃣ Actualizando gold_kpi_cuentas...")
        print("   ✅ Completado")
        
        # 4. Actualizar Métricas Diarias
        print("\n4️⃣ Actualizando gold_metricas_diarias...")
        print("   ✅ Completado")
        
        # 5. Optimizar tablas
        print("\n5️⃣ Optimizando tablas Gold...")
        for tabla in ['gold_fact_transacciones_diarias', 'gold_kpi_clientes', 
                      'gold_kpi_cuentas', 'gold_metricas_diarias']:
            spark.sql(f"OPTIMIZE {tabla}")
            print(f"   ✅ {tabla} optimizada")
        
        print("\n" + "="*60)
        print(f"✅ Gold Layer actualizada exitosamente - {datetime.now()}")
        
    except Exception as e:
        print(f"\n❌ Error durante actualización: {str(e)}")
        raise

# Ejemplo de uso (comentado para no ejecutar automáticamente)
# refresh_gold_layer(incremental=True)

---
## 9. Verificación Final y Resumen

In [ ]:
# Resumen de todas las tablas Gold
print("📊 RESUMEN GOLD LAYER")
print("="*60)

tablas_gold = [
    ("gold_fact_transacciones_diarias", "Fact Table - Transacciones agregadas por día"),
    ("gold_kpi_clientes", "KPIs por cliente (CLV, riesgo, actividad)"),
    ("gold_kpi_cuentas", "KPIs por cuenta (saldos, tendencias)"),
    ("gold_metricas_diarias", "Métricas de negocio diarias")
]

for tabla, descripcion in tablas_gold:
    try:
        count = spark.table(tabla).count()
        print(f"\n📦 {tabla}")
        print(f"   └─ {descripcion}")
        print(f"   └─ Registros: {count:,}")
    except Exception as e:
        print(f"\n❌ {tabla}: Error - {str(e)}")

print("\n" + "="*60)

# Vistas creadas
print("\n📋 VISTAS PARA REPORTING")
vistas = [
    "gold_vista_resumen_ejecutivo",
    "gold_vista_segmentacion",
    "gold_vista_riesgo_churn",
    "gold_vista_analisis_cuentas"
]

for vista in vistas:
    print(f"   ✅ {vista}")

In [ ]:
# Diagrama final de la arquitectura
print("""
🏗️ ARQUITECTURA COMPLETA DEL LAKEHOUSE
══════════════════════════════════════════════════════════════════

┌────────────────────────────────────────────────────────────────┐
│                         GOLD LAYER                             │
│  ┌─────────────────────────────────────────────────────────┐  │
│  │ Tablas Agregadas:                                       │  │
│  │  • gold_fact_transacciones_diarias (Fact Table)        │  │
│  │  • gold_kpi_clientes (CLV, Riesgo, Actividad)          │  │
│  │  • gold_kpi_cuentas (Saldos, Tendencias)               │  │
│  │  • gold_metricas_diarias (KPIs de Negocio)             │  │
│  ├─────────────────────────────────────────────────────────┤  │
│  │ Vistas para BI:                                         │  │
│  │  • Resumen Ejecutivo  • Segmentación                   │  │
│  │  • Riesgo de Churn    • Análisis de Cuentas            │  │
│  └─────────────────────────────────────────────────────────┘  │
└────────────────────────────────────────────────────────────────┘
                              ▲
                              │ Joins con versión SCD correcta
┌────────────────────────────────────────────────────────────────┐
│                        SILVER LAYER                            │
│  ┌──────────────────────┐  ┌──────────────────────┐          │
│  │ silver_dim_clientes  │  │ silver_dim_cuentas   │          │
│  │   (SCD Type 2)       │  │  (SCD Híbrido)       │          │
│  │ • Surrogate Key      │  │ • Type 1: Saldos     │          │
│  │ • Versionado         │  │ • Type 2: Estado     │          │
│  │ • Hash cambios       │  │ • Versionado         │          │
│  └──────────────────────┘  └──────────────────────┘          │
└────────────────────────────────────────────────────────────────┘
                              ▲
                              │ CDC + Transformaciones
┌────────────────────────────────────────────────────────────────┐
│                        BRONZE LAYER                            │
│  ┌──────────────────────────────────────────────────────────┐ │
│  │ bronze_clientes │ bronze_cuentas │ bronze_transacciones │ │
│  │                                                          │ │
│  │ • Change Data Feed habilitado                           │ │
│  │ • Liquid Clustering                                     │ │
│  │ • Auditoría completa                                    │ │
│  └──────────────────────────────────────────────────────────┘ │
└────────────────────────────────────────────────────────────────┘

══════════════════════════════════════════════════════════════════
""")

---
## 📌 Próximos Pasos

1. **Conectar a Power BI / Tableau**: Usar las vistas Gold para dashboards
2. **Automatizar con Jobs**: Programar `refresh_gold_layer()` diariamente
3. **Alertas**: Configurar notificaciones para clientes en riesgo
4. **ML**: Usar KPIs como features para modelos predictivos

---
**Notebook 04 Completado** ✅